In [9]:
import os
import numpy as np
import xarray as xr
import earthaccess
from datetime import datetime, timedelta
import pickle
import time
import random
from tqdm import tqdm

# Authenticate
auth = earthaccess.login(persist=True)

# Parameters
bbox = (-83.62, 41.34, -82, 42.27)      # (lon_min, lat_min, lon_max, lat_max)
res = 0.01
start_date = datetime(2024, 4, 14)
end_date = datetime(2025, 5, 23)
window_size = 7
decay = 0.8

# Ensure data directories exist
os.makedirs("../Data/", exist_ok=True)
os.makedirs("../Cache/", exist_ok=True)
os.makedirs("../Images/", exist_ok=True)

# Output grid
lat_bins = np.arange(bbox[1], bbox[3] + res, res)
lon_bins = np.arange(bbox[0], bbox[2] + res, res)
lat_centers = 0.5 * (lat_bins[:-1] + lat_bins[1:])
lon_centers = 0.5 * (lon_bins[:-1] + lon_bins[1:])
nlat, nlon = len(lat_centers), len(lon_centers)

# --- Retry Helpers ---

def safe_search(short_name, temporal, bounding_box, max_retries=500):
    retries = 0
    while True:
        try:
            results = earthaccess.search_data(
                short_name=short_name,
                temporal=temporal,
                bounding_box=bounding_box
            )
            return results
        except Exception as e:
            retries += 1
            if retries >= max_retries:
                print(f"Search failed after {max_retries} retries: {e}")
                return []
            wait = 5 + random.uniform(0, 3)
            print(f"Search error: {e}. Retrying in {wait:.1f} seconds...")
            time.sleep(wait)

def safe_download(results, directory="../Data/", max_retries=5):
    retries = 0
    while True:
        try:
            paths = earthaccess.download(results, directory)
            return paths
        except Exception as e:
            retries += 1
            if retries >= max_retries:
                print(f"Download failed after {max_retries} retries: {e}")
                return []
            wait = 5 + random.uniform(0, 3)
            print(f"Download error: {e}. Retrying in {wait:.1f} seconds...")
            time.sleep(wait)

# Wavelengths from a reference file
print("Retrieving wavelength list from a reference file...")
search_ref = safe_search(
    short_name="PACE_OCI_L2_AOP",
    temporal=("2024-06-01", "2024-06-05"),
    bounding_box=bbox,
)
if not search_ref:
    raise RuntimeError("No reference files found to retrieve wavelengths.")
ref_file = safe_download(search_ref, "../Data/")[0]
wave_all = xr.open_dataset(ref_file, group="sensor_band_parameters")["wavelength_3d"].data
num_channels = len(wave_all)
print(f"Found {num_channels} channels.")

# Prepare main array
total_days = (end_date - start_date).days + 1
ndarray_all = np.full((total_days, nlat, nlon, num_channels), np.nan, dtype=np.float32)

# Process day by day
for day_idx in range(total_days):
    current_date = start_date + timedelta(days=day_idx)
    window_start = current_date - timedelta(days=window_size)
    window_end = current_date - timedelta(days=1)

    print(f"\nProcessing {current_date.date()} (composite window {window_start.date()} to {window_end.date()})")

    # Search for data in window (with retry)
    results = safe_search(
        short_name="PACE_OCI_L2_AOP",
        temporal=(window_start.strftime("%Y-%m-%d"), window_end.strftime("%Y-%m-%d")),
        bounding_box=bbox,
    )

    if not results:
        print("No data found for this window. Skipping to next date.")
        continue

    # Download files (with retry)
    # Determine which files need downloading (those not yet cached)
    to_download = []
    for granule in results:
        granule_id = granule['umm']['DataGranule']['ArchiveAndDistributionInformation'][0]['Name']
        granule_basename = os.path.basename(granule_id)  # or appropriate key for ID
        cache_path = f"../Cache/{granule_basename}.nc.npz"
        if not os.path.exists(cache_path):
            to_download.append(granule)
    
    if to_download:
        print(f"Need to download {len(to_download)} new files for this window.")
        paths = safe_download(to_download, "../Data/")
        if not paths:
            print("No files downloaded for this window. Skipping to next date.")
            continue
    else:
        print("All files already cached. Skipping download.")
        paths = [f"../Data/{os.path.basename(granule['granule_id'])}" for granule in results]


    # Initialize sum and weight arrays
    sum_all = np.zeros((num_channels, nlat, nlon))
    weight_all = np.zeros((num_channels, nlat, nlon))

    for path in paths:
        base = os.path.basename(path)
        date_str = base.split(".")[1][:8]
        file_date = datetime.strptime(date_str, "%Y%m%d")
        delta_days = (window_end - file_date).days
        weight = decay ** delta_days if delta_days >= 0 else 0

        cache_file = f"../Cache/{base}.npz"

        if not os.path.exists(cache_file):
            print(f"Processing {base} (not in cache)")
            try:
                nav = xr.open_dataset(path, group="navigation_data")
                lat = nav["latitude"].values
                lon = nav["longitude"].values

                rrs_ds = xr.open_dataset(path, group="geophysical_data")["Rrs"]
                rrs_ds = rrs_ds.assign_coords(wavelength_3d=wave_all)

                lat_idx_all = []
                lon_idx_all = []
                ch_idx_all = []
                val_all = []

                for ch_idx, wl in tqdm(list(enumerate(wave_all)), desc=f"Channels in {base}", leave=False):
                    band = rrs_ds.sel(wavelength_3d=wl, method="nearest").values
                    mask = (
                        np.isfinite(band) &
                        (lat >= bbox[1]) & (lat <= bbox[3]) &
                        (lon >= bbox[0]) & (lon <= bbox[2])
                    )
                    lat_valid = lat[mask]
                    lon_valid = lon[mask]
                    val_valid = band[mask]

                    lat_idx = np.searchsorted(lat_bins, lat_valid) - 1
                    lon_idx = np.searchsorted(lon_bins, lon_valid) - 1

                    lat_idx_all.extend(lat_idx)
                    lon_idx_all.extend(lon_idx)
                    ch_idx_all.extend([ch_idx] * len(val_valid))
                    val_all.extend(val_valid)

                np.savez_compressed(cache_file,
                                    lat_idx=np.array(lat_idx_all, dtype=np.int16),
                                    lon_idx=np.array(lon_idx_all, dtype=np.int16),
                                    ch_idx=np.array(ch_idx_all, dtype=np.int16),
                                    val=np.array(val_all, dtype=np.float32))
                print(f"Cached data to {cache_file}")

            except Exception as e:
                print(f"Failed to process {path}: {e}")
                continue
        else:
            print(f"Using cached data for {base}")

        # Load from cache
        data = np.load(cache_file)
        lat_idx = data['lat_idx']
        lon_idx = data['lon_idx']
        ch_idx = data['ch_idx']
        val = data['val']

        for j in range(len(val)):
            if 0 <= lat_idx[j] < nlat and 0 <= lon_idx[j] < nlon:
                sum_all[ch_idx[j], lat_idx[j], lon_idx[j]] += val[j] * weight
                weight_all[ch_idx[j], lat_idx[j], lon_idx[j]] += weight

    # Finalize average
    with np.errstate(invalid="ignore", divide="ignore"):
        avg_all = sum_all / weight_all
        avg_all[weight_all == 0] = np.nan

    ndarray_all[day_idx] = np.transpose(avg_all, (1, 2, 0))

    # Delete old files
    delete_date = (current_date - timedelta(days=window_size)).strftime('%Y%m%d')
    for fname in os.listdir("../Data/"):
        if delete_date in fname and fname.endswith(".nc"):
            try:
                os.remove(os.path.join("../Data/", fname))
                print(f"Deleted old file: {fname}")
            except Exception as e:
                print(f"Could not delete {fname}: {e}")

# Save results
np.save("../Images/composite_data.npy", ndarray_all)
print("\nSaved full 4D composite data array to '../Images/composite_data.npy'.")

metadata = {"wavelengths": wave_all, "lat": lat_centers, "lon": lon_centers}
with open("../Images/composite_metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)
print("Saved metadata (wavelengths, lat, lon) to '../Images/composite_metadata.pkl'.")


Retrieving wavelength list from a reference file...


QUEUEING TASKS | : 100%|████████████████████████| 9/9 [00:00<00:00, 3115.35it/s]
PROCESSING TASKS | : 100%|█████████████████████| 9/9 [00:00<00:00, 29308.02it/s]
COLLECTING RESULTS | : 100%|███████████████████| 9/9 [00:00<00:00, 49474.10it/s]


Found 172 channels.

Processing 2024-04-14 (composite window 2024-04-07 to 2024-04-13)
Need to download 10 new files for this window.


QUEUEING TASKS | : 100%|██████████████████████| 10/10 [00:00<00:00, 1127.38it/s]
PROCESSING TASKS | : 100%|██████████████████████| 10/10 [00:29<00:00,  2.92s/it]
COLLECTING RESULTS | : 100%|█████████████████| 10/10 [00:00<00:00, 67541.13it/s]


Using cached data for PACE_OCI.20240407T174325.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240408T181826.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240409T171508.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240409T185328.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240410T175010.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240411T164652.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240411T182512.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240412T172154.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240412T190014.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240413T175656.L2.OC_AOP.V3_0.nc
Deleted old file: PACE_OCI.20240407T174325.L2.OC_AOP.V3_0.nc

Processing 2024-04-15 (composite window 2024-04-08 to 2024-04-14)
Need to download 11 new files for this window.


...

Using cached data for PACE_OCI.20240412T172154.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240412T190014.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240413T175656.L2.OC_AOP.V3_0.nc


Using cached data for PACE_OCI.20240414T165338.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240414T183158.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240415T172837.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240415T190657.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240416T180338.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240417T170019.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240417T183839.L2.OC_AOP.V3_0.nc
Using cached data for PACE_OCI.20240418T173520.L2.OC_AOP.V3_0.nc
Deleted old file: PACE_OCI.20240412T190014.L2.OC_AOP.V3_0.nc
Deleted old file: PACE_OCI.20240412T172154.L2.OC_AOP.V3_0.nc

Processing 2024-04-20 (composite window 2024-04-13 to 2024-04-19)
Need to download 10 new files for this window.


QUEUEING TASKS | : 100%|██████████████████████| 10/10 [00:00<00:00, 3425.88it/s]
PROCESSING TASKS | : 100%|███████████████████| 10/10 [00:00<00:00, 44858.87it/s]
COLLECTING RESULTS | : 100%|█████████████████| 10/10 [00:00<00:00, 71943.46it/s]

Using cached data for PACE_OCI.20240413T175656.L2.OC_AOP.V3_0.nc


KeyboardInterrupt: 

In [7]:
import os
import numpy as np
import xarray as xr
import earthaccess
from datetime import datetime, timedelta
import pickle
import time
import random
from tqdm import tqdm

# Authenticate
auth = earthaccess.login(persist=True)

# Parameters
bbox = (-83.62, 41.34, -82, 42.27)      # (lon_min, lat_min, lon_max, lat_max)
res = 0.01
start_date = datetime(2024, 4, 14)
end_date = datetime(2025, 5, 23)
window_size = 7
decay = 0.8

# Ensure data directories exist
os.makedirs("../Data/", exist_ok=True)
os.makedirs("../Cache/", exist_ok=True)
os.makedirs("../Images/", exist_ok=True)

# Output grid
lat_bins = np.arange(bbox[1], bbox[3] + res, res)
lon_bins = np.arange(bbox[0], bbox[2] + res, res)
lat_centers = 0.5 * (lat_bins[:-1] + lat_bins[1:])
lon_centers = 0.5 * (lon_bins[:-1] + lon_bins[1:])
nlat, nlon = len(lat_centers), len(lon_centers)

# --- Retry Helpers ---

def safe_search(short_name, temporal, bounding_box, max_retries=500):
    retries = 0
    while True:
        try:
            results = earthaccess.search_data(
                short_name=short_name,
                temporal=temporal,
                bounding_box=bounding_box
            )
            return results
        except Exception as e:
            retries += 1
            if retries >= max_retries:
                print(f"Search failed after {max_retries} retries: {e}")
                return []
            wait = 5 + random.uniform(0, 3)
            print(f"Search error: {e}. Retrying in {wait:.1f} seconds...")
            time.sleep(wait)
            
results = safe_search(
        short_name="PACE_OCI_L2_AOP",
        temporal=("2024-06-01", "2024-06-05"),
        bounding_box=bbox,
    )
if results:
    print("\nFirst granule keys:")
    print(results[0]['umm'].keys())
    print("\nFirst granule example:")
    import pprint
    pprint.pprint(results[0]['umm'])
    print("\nPossible filename:", )
else:
    print("No results found.")


First granule keys:
dict_keys(['TemporalExtent', 'GranuleUR', 'SpatialExtent', 'ProviderDates', 'CollectionReference', 'RelatedUrls', 'CloudCover', 'DataGranule', 'Platforms', 'MetadataSpecification'])

First granule example:
{'CloudCover': 48.2,
 'CollectionReference': {'ShortName': 'PACE_OCI_L2_AOP', 'Version': '3.0'},
 'DataGranule': {'ArchiveAndDistributionInformation': [{'Format': 'netCDF-4',
                                                        'MimeType': 'application/x-netcdf',
                                                        'Name': 'PACE_OCI.20240601T165222.L2.OC_AOP.V3_0.nc',
                                                        'SizeInBytes': 397593763}],
                 'DayNightFlag': 'Day',
                 'Identifiers': [{'Identifier': 'PACE_OCI.20240601T165222.L2.OC_AOP.V3_0.nc',
                                  'IdentifierType': 'ProducerGranuleId'}],
                 'ProductionDateTime': '2025-02-10T19:48:27.503Z'},
 'GranuleUR': 'PACE_OCI_L2_AOP_PACE

In [ ]:
wave = xr.open_dataset(paths[0], group="sensor_band_parameters")["wavelength_3d"].data
indices = np.where(wave == 450)
indices[0]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

def generate_day_images(n, r_idx=113, g_idx=84, b_idx=42):
    """
    Generate individual true-color images for the first n days in the composite data.
    
    Args:
        n (int): Number of days to generate images for.
        r_idx (int): Index of the wavelength to use for red channel.
        g_idx (int): Index of the wavelength to use for green channel.
        b_idx (int): Index of the wavelength to use for blue channel.
    """
    # Load data
    data_path = "../Images/composite_data.npy"
    meta_path = "../Images/composite_metadata.pkl"
    if not os.path.exists(data_path) or not os.path.exists(meta_path):
        print("Required files not found. Run the composite script first.")
        return
    
    data = np.load(data_path)
    print("data.shape", data.shape)
    with open(meta_path, "rb") as f:
        meta = pickle.load(f)
    
    lat = meta["lat"]
    lon = meta["lon"]
    wavelengths = meta["wavelengths"]

    num_days = data.shape[0]
    n = min(n, num_days)  # Ensure n doesn't exceed available data

    # Normalize Rrs data (assumed 0-0.03)
    def normalize(arr, vmin=0, vmax=0.03):
        return np.clip((arr - vmin) / (vmax - vmin), 0, 1)

    for day_idx in range(n):
        daily_data = data[day_idx]  # shape (h, w, c)

        if np.isnan(daily_data).all():
            print(f"Day {day_idx + 1} has no valid data. Skipping.")
            continue

        r = normalize(daily_data[:, :, r_idx])
        g = normalize(daily_data[:, :, g_idx])
        b = normalize(daily_data[:, :, b_idx])

        rgb = np.stack([r, g, b], axis=-1)

        # Plot
        plt.figure(figsize=(10, 6))
        plt.imshow(rgb, origin="lower", extent=[lon.min(), lon.max(), lat.min(), lat.max()])
        plt.title(f"True-Color Image - Day {day_idx + 1}")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.tight_layout()

        out_path = f"../Images/dayyy_{day_idx + 1:03d}.png"
        plt.savefig(out_path)
        plt.close()
        print(f"Saved image: {out_path}")

    return data


In [ ]:
data = generate_day_images(5)

In [ ]:
data[3][45][100]

In [ ]:
break

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import earthaccess
from datetime import datetime, timedelta

# Authenticate
auth = earthaccess.login(persist=True)

# Parameters
selected_wavelengths = [645, 555, 450]  # R, G, B
bbox = (-83.62, 41.34, -82, 42.27)      # (lon_min, lat_min, lon_max, lat_max)
res = 0.01  # grid resolution in degrees
start_date = datetime(2024, 5, 17)
end_date = datetime(2025, 5, 23)

# Ensure output directories exist
os.makedirs("../Images/", exist_ok=True)
os.makedirs("../Data/", exist_ok=True)

# Output grid
lat_bins = np.arange(bbox[1], bbox[3] + res, res)
lon_bins = np.arange(bbox[0], bbox[2] + res, res)
lat_centers = 0.5 * (lat_bins[:-1] + lat_bins[1:])
lon_centers = 0.5 * (lon_bins[:-1] + lon_bins[1:])
nlat, nlon = len(lat_centers), len(lon_centers)

# Iterate through each day
current_date = start_date
while current_date <= end_date:
    window_start = current_date - timedelta(days=4)
    window_end = current_date
    print(f"Processing window: {window_start.date()} to {window_end.date()}")

    results = earthaccess.search_data(
        short_name="PACE_OCI_L2_AOP",
        temporal=(window_start.strftime("%Y-%m-%d"), window_end.strftime("%Y-%m-%d")),
        bounding_box=bbox,
    )

    if not results:
        print(f"No data for {window_end.strftime('%Y-%m-%d')}. Skipping.")
        current_date += timedelta(days=1)
        continue

    paths = earthaccess.download(results, "../Data/")

    if not paths:
        print(f"No files downloaded for {window_end.strftime('%Y-%m-%d')}. Skipping.")
        current_date += timedelta(days=1)
        continue

    # Initialize sum and count arrays
    sum_rgb = np.zeros((3, nlat, nlon))
    count_rgb = np.zeros((3, nlat, nlon))

    try:
        wave = xr.open_dataset(paths[0], group="sensor_band_parameters")["wavelength_3d"].data
    except Exception as e:
        print(f"Failed to read wavelength data: {e}")
        current_date += timedelta(days=1)
        continue

    # Process each file
    for path in paths:
        print(f"Processing {path}")
        try:
            rrs_ds = xr.open_dataset(path, group="geophysical_data")["Rrs"]
            rrs_ds = rrs_ds.assign_coords(wavelength_3d=wave)

            nav = xr.open_dataset(path, group="navigation_data")
            lat = nav["latitude"].values
            lon = nav["longitude"].values

            for b, wl in enumerate(selected_wavelengths):
                band = rrs_ds.sel(wavelength_3d=wl, method="nearest").values
                mask = (
                    np.isfinite(band) &
                    (lat >= bbox[1]) & (lat <= bbox[3]) &
                    (lon >= bbox[0]) & (lon <= bbox[2])
                )

                lat_valid = lat[mask]
                lon_valid = lon[mask]
                val_valid = band[mask]

                lat_idx = np.searchsorted(lat_bins, lat_valid) - 1
                lon_idx = np.searchsorted(lon_bins, lon_valid) - 1

                for j in range(len(val_valid)):
                    if 0 <= lat_idx[j] < nlat and 0 <= lon_idx[j] < nlon:
                        sum_rgb[b, lat_idx[j], lon_idx[j]] += val_valid[j]
                        count_rgb[b, lat_idx[j], lon_idx[j]] += 1
        except Exception as e:
            print(f"Failed to process {path}: {e}")

    # Compute mean reflectance
    with np.errstate(invalid='ignore', divide='ignore'):
        mean_rgb = sum_rgb / count_rgb
        mean_rgb = np.nan_to_num(mean_rgb, nan=0.0)

    # Normalize reflectance for display (Rrs units are ~0–0.03)
    def normalize(arr, vmin=0, vmax=0.03):
        return np.clip((arr - vmin) / (vmax - vmin), 0, 1)

    r = normalize(mean_rgb[0])
    g = normalize(mean_rgb[1])
    b = normalize(mean_rgb[2])
    rgb = np.stack([r, g, b], axis=-1)

    # Save true color image
    plt.figure(figsize=(10, 6))
    plt.imshow(rgb, origin="lower", extent=[bbox[0], bbox[2], bbox[1], bbox[3]])
    plt.title(f"5-Day Composite Ending {window_end.strftime('%Y-%m-%d')}")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.tight_layout()
    out_path = f"../Images/{window_end.strftime('%Y%m%d')}.png"
    plt.savefig(out_path)
    plt.close()
    print(f"Saved image: {out_path}")

    # Clean up only the earliest date in window
    delete_date = window_start.strftime('%Y%m%d')
    for fname in os.listdir("../Data/"):
        if delete_date in fname and fname.endswith(".nc"):
            try:
                os.remove(os.path.join("../Data/", fname))
                print(f"Deleted old file: {fname}")
            except Exception as e:
                print(f"Could not delete {fname}: {e}")

    current_date += timedelta(days=1)
